In [1]:
import pandas as pd
import requests

In [ ]:
# http://alegeri.roaep.ro/?alegeri=prezidentiale-2009
# https://webspace.ulbsibiu.ro/andrei.gheorghita/documents/a11.pdf

In [ ]:
# "http://alegeri.roaep.ro/wp-content/plugins/aep/aep_data.php?name=v1_prez_Judet_Lista&parameter=261"

# "http://alegeri.roaep.ro/wp-content/plugins/aep/aep_data.php?name=v1_prez_Localitate_Lista&parameter=261&parameter=1"

# "http://alegeri.roaep.ro/wp-content/plugins/aep/aep_data.php?name=v1_prez_Sectie_Lista&parameter=261&parameter=87314"


In [25]:
election_id = 261
year = 2009
API = "http://alegeri.roaep.ro/wp-content/plugins/aep/aep_data.php"

data_county = requests.get(f"{API}?name=v1_prez_Judet_Lista&parameter={election_id}").json()
df_county = pd.DataFrame(data_county)
df_county.head()

,COD_JUD,DEN_JUD
0,1,Alba
1,2,Arad
2,3,Arges
3,4,Bacau
4,5,Bihor


In [21]:
output = []
for item in data_county:
    county_id = item["COD_JUD"]
    tmp = requests.get(f"{API}?name=v1_prez_Localitate_Lista&parameter={parameter}&parameter={county_id}").json()
    df_tmp = pd.DataFrame(tmp)
    df_tmp["county_id"] = county_id
    output.append(df_tmp)

df_location = pd.concat(output, ignore_index=True)
df_location

,SIRUTA,Nume,Id,county_id
0,87312,ALBAC,87312,1
1,87313,ALMASU MARE,87313,1
2,87314,ARIESENI,87314,1
3,87315,AVRAM IANCU,87315,1
4,87317,BERGHIN,87317,1
...,...,...,...,...
3478,90410,VIDRA,90410,41
3479,90411,VINTILEASCA,90411,41
3480,90412,VIZANTEA-LIVEZI,90412,41
3481,90415,VRANCIOAIA,90415,41


In [19]:
output = []
for location_id, _ in df_location.groupby("Id"):
    data = requests.get(f"{API}?name=v1_prez_Sectie_Lista&parameter={election_id}&parameter={location_id}").json()
    
    tmp = pd.DataFrame(data)
    df_tmp = pd.DataFrame(tmp)
    df_tmp["location_id"] = location_id
    output.append(df_tmp)
    
df_polling = pd.concat(output, ignore_index=True)
df_polling

,NSV,Adresa,location_id
0,2,CAPE TOWN,87008
1,1,PRETORIA,87009
2,3,TIRANA,87010
3,4,ALGER,87011
4,5,LUANDA,87012
...,...,...,...
21701,117,SCOALA NICOLAE PETRESCU (fosta scoala generala...,90489
21702,115,SCOALA GENERALA CU CLASELE I-VIII BARBULESTI,90490
21703,116,SCOALA GENERALA CU CLASELE I-VIII BARBULESTI,90490
21704,117,SCOALA GENERALA CU CLASELE I-VIII BARBULESTI,90490


In [43]:
df_geo = pd.merge(df_location, df_polling, left_on="Id", right_on="location_id")
df_geo = pd.merge(df_geo, df_county, left_on="county_id", right_on="COD_JUD")
df_geo["election_id"] = election_id

df_geo.to_csv(f"data/Romania/{year}_polling_stations.csv", index=False)
df_geo

,SIRUTA,Nume,Id,county_id,NSV,Adresa,location_id,COD_JUD,DEN_JUD,election_id
0,87312,ALBAC,87312,1,193,"Caminul cultural Albac, strada Closca, nr. 9, ...",87312,1,Alba,261
1,87312,ALBAC,87312,1,194,"Scoala cu clasele I - IV Cionesti, sat. Ciones...",87312,1,Alba,261
2,87312,ALBAC,87312,1,195,"Scoala cu clasele I - IV Costesti, sat. Costes...",87312,1,Alba,261
3,87312,ALBAC,87312,1,196,"Scoala cu clasele I - IV Rogoz, sat. Rogoz, Sa...",87312,1,Alba,261
4,87312,ALBAC,87312,1,197,"Scoala cu clasele I - IV Potionci, sat. Potion...",87312,1,Alba,261
...,...,...,...,...,...,...,...,...,...,...
21701,90416,VULTURU,90416,41,426,BOTIRLAU SCOALA GENERALA,90416,41,Vrancea,261
21702,90416,VULTURU,90416,41,427,HINGULESTI SCOALA GENERALA,90416,41,Vrancea,261
21703,90416,VULTURU,90416,41,428,MALURI SCOALA,90416,41,Vrancea,261
21704,90416,VULTURU,90416,41,429,VADU ROSCA SCOALA,90416,41,Vrancea,261


In [5]:
requests.get("http://alegeri.roaep.ro/wp-content/plugins/aep/aep_data.php?name=v1_prez_Sectie_Lista&parameter=261&parameter=87314").json()

[{'NSV': 205,
  'Adresa': 'GRUP SCOLAR DE TURISM ARIESENI, Sala clasei I-a Tel:0258 779 066'},
 {'NSV': 206, 'Adresa': 'SCOALA CU CLASELE I-IV Casa de Piatra, Sala mare'},
 {'NSV': 207, 'Adresa': 'SCOALA CU CLASELE I-IV Cobles, Sala mare'},
 {'NSV': 208,
  'Adresa': 'GRUP SCOLAR DE TURISM ARIESENI, sat arieseni, nr.8, Sala clasei  a IV-a Tel:0258 779 066'}]

In [31]:
df_geo

,SIRUTA,Nume,Id,county_id,NSV,Adresa,location_id,COD_JUD,DEN_JUD
0,87312,ALBAC,87312,1,193,"Caminul cultural Albac, strada Closca, nr. 9, ...",87312,1,Alba
1,87312,ALBAC,87312,1,194,"Scoala cu clasele I - IV Cionesti, sat. Ciones...",87312,1,Alba
2,87312,ALBAC,87312,1,195,"Scoala cu clasele I - IV Costesti, sat. Costes...",87312,1,Alba
3,87312,ALBAC,87312,1,196,"Scoala cu clasele I - IV Rogoz, sat. Rogoz, Sa...",87312,1,Alba
4,87312,ALBAC,87312,1,197,"Scoala cu clasele I - IV Potionci, sat. Potion...",87312,1,Alba
...,...,...,...,...,...,...,...,...,...
21701,90416,VULTURU,90416,41,426,BOTIRLAU SCOALA GENERALA,90416,41,Vrancea
21702,90416,VULTURU,90416,41,427,HINGULESTI SCOALA GENERALA,90416,41,Vrancea
21703,90416,VULTURU,90416,41,428,MALURI SCOALA,90416,41,Vrancea
21704,90416,VULTURU,90416,41,429,VADU ROSCA SCOALA,90416,41,Vrancea


In [35]:
output = []
for idx, df_tmp in df_geo.groupby(["county_id", "NSV"]):
    county_id, polling_id = idx
    
    data = requests.get(f"{API}?name=v1_prez_Sectie_Voturi&parameter={election_id}&parameter={county_id}&parameter={polling_id}").json()
    df = pd.DataFrame(data)
    df["polling_id"] = polling_id
    df["county_id"] = county_id
    df["election_id"] = election_id
    
    output.append(df)

In [37]:
df = pd.concat(output, ignore_index=True)

In [38]:
df.to_csv(f"data/Romania/{year}_votes.csv", index=False)

In [39]:
df

,Candidat,AfilierePolitica,Voturi,Procent,NumarTur,polling_id,county_id,election_id
0,Traian Băsescu,PDL,493,53.64,2,1,1,261
1,Mircea Geoană,PSD + PC,426,46.35,2,1,1,261
2,Traian Băsescu,PDL,328,35.88,1,1,1,261
3,Crin Antonescu,PNL,282,30.85,1,1,1,261
4,Mircea Geoană,PSD + PC,230,25.16,1,1,1,261
...,...,...,...,...,...,...,...,...
247580,Crin Antonescu,PNL,48,8.33,1,294,48,261
247581,George Becali,PNG - CD,20,3.47,1,294,48,261
247582,Corneliu Vadim Tudor,PRM,19,3.29,1,294,48,261
247583,Remus Cernea,PV,2,0.34,1,294,48,261


In [11]:
county_id = 1
polling_id = 206

data = requests.get(f"{API}?name=v1_prez_Sectie_Voturi&parameter={election_id}&parameter={county_id}&parameter={polling_id}").json()
df = pd.DataFrame(data)
df["polling_id"] = polling_id
df["county_id"] = county_id
df["election_id"] = election_id

df

,Candidat,AfilierePolitica,Voturi,Procent,NumarTur,polling_id,county_id,election_id
0,Traian Băsescu,PDL,58,69.87,2,206,1,261
1,Traian Băsescu,PDL,46,63.01,1,206,1,261
2,Mircea Geoană,PSD + PC,25,30.12,2,206,1,261
3,Crin Antonescu,PNL,20,27.39,1,206,1,261
4,Mircea Geoană,PSD + PC,4,5.47,1,206,1,261
5,George Becali,PNG - CD,2,2.73,1,206,1,261
6,Corneliu Vadim Tudor,PRM,1,1.36,1,206,1,261


In [ ]:
# Summary from the Polling Station

requests.get(f"{API}?name=v1_prez_Sectie_Sumar&parameter={election_id}&parameter={county_id}&parameter={polling_id}")